In [1]:
import pandas as pd
df_regions = pd.read_csv("Data/noc_regions.csv")
df_events = pd.read_csv("Data/athlete_events.csv")

In [2]:
df_regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [3]:
df_events.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## A) 
Hur många länder som är med i datan?

In [4]:
number_od_countries = df_regions["NOC"].nunique()
print(f"Det finns {number_od_countries} olika länder i datan.")

Det finns 230 olika länder i datan.


## B)
Vilka länder är med? (förkortningarna räcker)

In [5]:
countries = df_regions["NOC"]
countries

0      AFG
1      AHO
2      ALB
3      ALG
4      AND
      ... 
225    YEM
226    YMD
227    YUG
228    ZAM
229    ZIM
Name: NOC, Length: 230, dtype: object

## C) 
Vilka sporter är med?

In [6]:
df_events["Sport"]

0            Basketball
1                  Judo
2              Football
3            Tug-Of-War
4         Speed Skating
              ...      
271111             Luge
271112      Ski Jumping
271113      Ski Jumping
271114        Bobsleigh
271115        Bobsleigh
Name: Sport, Length: 271116, dtype: object

## D)
Vilka medaljtyper finns det?

In [7]:
unique_medals = df_events['Medal'].dropna().unique()
medal_string = ', '.join(unique_medals[:-1]) + f" och {unique_medals[-1]}"
print(f"Medaljtyperna är: {medal_string}.")

Medaljtyperna är: Gold, Bronze och Silver.


## E)
Ta reda på statistik för åldern: medelvärde, median, min, max, standardavvikelse

In [8]:
df_events['Age'].agg(
    Medelvärde='mean', 
    Median='median', 
    Minimum='min', 
    Maximum='max', 
    Standardavvikelse='std'
)

Medelvärde           25.556898
Median               24.000000
Minimum              10.000000
Maximum              97.000000
Standardavvikelse     6.393561
Name: Age, dtype: float64

## F)
Utforska datan vidare med egna frågor
- Vilka åldersintervaller tar flest medaljer?
- Varierar medelåldern mellan mästerskapen? 
- Könsfördelning mellan mästerskapen?
- Eller andra intressanta grejer...

## G)
Diagram över könsfördelningen

In [30]:
import plotly.express as px
df_events_gh = df_events.copy()

In [43]:
fig = px.pie(df_events_gh, "Sex",
             title="Könsfördelning: M=Man, F=Kvinna", 
             template="seaborn", 
             labels=dict(Sex="Kön"))

fig.update_layout(title_x = 0.5)
fig.show()

## H)
Diagram över topp 10 länder som tagit flest medaljer

In [46]:
# Group by countries and count all the medals taken by each contry
countries = df_events_gh.groupby("Team").agg({"Medal": "count"}).sort_values(by="Medal", ascending=False)

fig = px.bar(countries.head(10), y="Medal",
             template="seaborn",
             labels=dict(Medal = "Antal medaljer", Team = "Land"), 
             title="Antal medaljer/land (top 10)")

fig.show()



## I) 
Plotta gärna fler saker som är intressant. *(förslagsvis från F)*